# Prompt Decomposition Example
This notebook contains examples of prompt decomposition, or taking one long prompt or task and breaking it down into smaller parts.  These smaller parts can then be run independantly, often in parallel, and often on smaller models.  This can lead to significant performance enhancements and cost savings, may increase quality, and will make your prompts much easier to maintain as the workload grows.  This is because each step can be maintained and tested independantly, and can run on different models.  You can also alter any one step without impacting all the others.

Here are three common times when Prompt Decomposition can be helpful:

  1)  Breaking multi-step prompts into a maintainable DAG.  This can be helpful when a prompt reads like code, with a large number of steps or if/then instructions.  Instead, consider breaking these out and generating a flow diagram, resulting in maintainable individual pieces.
  2)  Streamlining long linear prompts.  Even where prompts have only a single logical flow, if they are long, it can help to break the flow into small, sequential steps.  These steps combined may execute faster than the long original prompt, and they can also be maintained and tested independently. 
  3) Preprocessing of RAG or context data.  For example, if context data is large, such as a long support document, consider a prompt that summarizes that content once, then future queries (from the same or different conversations) retrieve the cached summary rather than the long document. 
  
This notebook follows this structure:
1) Set up the envionment
    * Functions for connecting to Bedrock, including multithreading
    * Evaluation functions, for measuring the accuracy of each prompt
2) Examples of decomposition
    * Task based decomposition
    * Volume based decomposition

# 1) Set up the envionment

In [6]:
#for connecting with Bedrock, use Boto3
import boto3, time, json
from botocore.config import Config

#increase the standard time out limits in boto3, because Bedrock may take a while to respond to large requests.
my_config = Config(
    connect_timeout=60*5,
    read_timeout=60*5,
)
bedrock = boto3.client(service_name='bedrock-runtime',config=my_config)
bedrock_service = boto3.client(service_name='bedrock',config=my_config)

In [7]:
#check that it's working:
models = bedrock_service.list_foundation_models()
for line in models["modelSummaries"]:
    #print (line["modelId"])
    pass
if "anthropic.claude-3" in str(models):
    print("Claud-v3 found!")
else:
    print ("Error, no model found.")

Claud-v3 found!


In [8]:
#helper function for converting tokens to public pricing for Claude.
input_token_haiku = 0.25/1000000
output_token_haiku = 1.25/1000000
input_token_sonnet = 3.00/1000000
output_token_sonnet = 15.00/1000000
input_token_opus = 15.00/1000000
output_token_opus = 75.00/1000000
def calculate_cost(usage, model):
    '''
    Takes the usage tokens returned by Bedrock in input and output, and coverts to cost in dollars.
    '''
    cost = 0
    if model=='haiku':
        cost+= usage['input_tokens']*input_token_haiku
        cost+= usage['output_tokens']*output_token_haiku
    if model=='sonnet':
        cost+= usage['input_tokens']*input_token_sonnet
        cost+= usage['output_tokens']*output_token_sonnet
    if model=='opus':
        cost+= usage['input_tokens']*input_token_opus
        cost+= usage['output_tokens']*output_token_opus
    return cost

In [290]:
MAX_ATTEMPTS = 1 #how many times to retry if Claude is not working.
session_cache = {} #all calls are stored in the local cache.  This means subsequent calls using the same prompt will return instantly with the cached results, rather than hit Bedrock again. 
def ask_claude(messages,system="", model="sonnet", ignore_cache=False, DEBUG=False):
    '''
    Send a prompt to Bedrock, and return the response.
    messages can be an array of role/message pairs, or a string.
    DEBUG is used to see exactly what is being sent to and from Bedrock.
    model can be haiku or sonnet
    Set ignore_cache to True if you want to force a new call to Bedrock
    '''
    raw_prompt_text = str(messages)
    #if the messages are just a string, convert to the Messages API format.
    if type(messages)==str:
        messages = [{"role": "user", "content": messages}]
    
    #build the JSON to send to Bedrock
    prompt_json = {
        "system":system,
        "messages": messages,
        "max_tokens": 4096, # 4096 is a hard limit to output length in Claude 3
        "temperature": 0.1, #creativity on a scale from 0-1.
        "anthropic_version":"",
        "top_k": 250,
        "top_p": 0.7,
        "stop_sequences": ["\n\nHuman:"]
    }
    
    if DEBUG: print("Sending:\nSystem:\n",system,"\nMessages:\n",str(messages))
    
    #pick the correct endpoint for the model we want to use.
    if model== "opus":
        modelId = 'error'
    elif model== "sonnet":
        modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
    elif model== "haiku":
        modelId = 'anthropic.claude-3-haiku-20240307-v1:0'
    else:
        print ("ERROR:  Bad model, must be opus, sonnet, or haiku.")
        modelId = 'error'
    
    #use the json+modelID to identify unique calls to Bedrock.
    raw_prompt_json = str(prompt_json)+modelId
    
    #if this is already in the cashe, return data from the cache and skip Bedrock.
    if raw_prompt_json in session_cache and not ignore_cache:
        if DEBUG: print ("Using results from cache, skipping Bedrock.")
        cached = session_cache[raw_prompt_json]
        return [raw_prompt_text,cached[0],cached[1],cached[2],cached[3]]
    
    attempt = 1
    query_time = -1
    usage = (-1,-1)
    while True:
        try:
            start_time = time.time()
            response = bedrock.invoke_model(body=json.dumps(prompt_json), modelId=modelId, accept='application/json', contentType='application/json')
            response_body = json.loads(response.get('body').read())
            #print (response_body)
            results = response_body.get("content")[0].get("text")
            usage = response_body.get("usage")
            query_time = round(time.time()-start_time,2)
            if DEBUG:print("Recieved:",results)
            break
        except Exception as e:
            print("Error with calling Bedrock: "+str(e))
            attempt+=1
            if attempt>MAX_ATTEMPTS:
                print("Max attempts reached!")
                results = str(e)
                break
            else:#retry in 10 seconds
                time.sleep(10)
    session_cache[raw_prompt_json] = [results,usage,query_time,system]
    return [raw_prompt_text,results,usage,query_time,system]

In [10]:
#check that it's working:
try:
    query = "Please say the number four."
    system = "You always reply in Spanish."
    result = ask_claude(query,system=system,ignore_cache=True,DEBUG=False)
    print("System Instructions: ",result[4])
    print("Prompt: ",query)
    print("Response: ",result[1])
    print(result[2])
    print("Query time: ",result[3],"seconds")
    #check that cost calculation is working
    cost = calculate_cost(result[2], 'haiku')
    print("Cost for running this query 1 million times: $",cost*1000000)
except Exception as e:
    print("Error with calling Claude: "+str(e))

System Instructions:  You always reply in Spanish.
Prompt:  Please say the number four.
Response:  Cuatro.
{'input_tokens': 19, 'output_tokens': 7}
Query time:  0.43 seconds
Cost for running this query 1 million times: $ 13.5


#### Now that we have set up basic Claude connectivity, let's create a helper function for multi-threading.  This will allow us to run our evaluation tests in parallel.
Note that Bedrock has both a requests per minute and a token per limit maximum at the account level, so make sure you don't hit that max while testing.

In [11]:
from queue import Queue
from threading import Thread

# Threading function for queue processing.
def thread_request(q, result):
    while not q.empty():
        work = q.get()    #fetch new work from the Queue
        try:
            unique_id,prompt = work[1]
            data = ask_claude(prompt,system=work[2],model=work[3],ignore_cache=work[4])
            result[work[0]] = (unique_id,data)  #Store data back at correct index
        except Exception as e:
            print('Error with prompt!',str(e))
            result[work[0]] = (str(e))
        #signal to the queue that task has been processed
        q.task_done()
    return True

def ask_claude_threaded(prompts,system="",model="sonnet",ignore_cache=False):
    '''
    Call ask_claude, but multi-threaded.
    prompts should be a tuple of (unique_request_ID, prompt)
    unique_request_ID should be a unique integer, and is used to distingish prompts if the same one is sent more than once.  (sometimes a vaild request because LLM's are non-deterministic)
    Returns a dict of the prompts and responces.
    '''
    q = Queue(maxsize=0)
    num_theads = min(50, len(prompts))
    #Populating Queue with tasks
    results = [{} for x in prompts];
    #load up the queue with the promts to fetch and the index for each job (as a tuple):
    for i in range(len(prompts)):
        #need the index and the url in each queue item.
        q.put((i,prompts[i],system,model,ignore_cache))
        
    #Starting worker threads on queue processing
    for i in range(num_theads):
        #print('Starting thread ', i)
        worker = Thread(target=thread_request, args=(q,results))
        worker.daemon = True
        worker.start()

    #now we wait until the queue has been processed
    q.join()
    return results

In [12]:
%%time
#test if our threaded Claude calls are working
q1 = (1,[{"role": "user", "content": "Please say the number one."}])
q2 = (2,[{"role": "user", "content": "Please say the number two."}])
q3 = (3,[{"role": "user", "content": "Please say the number three."}])
results = ask_claude_threaded([q1,q2,q3],system="you only reply in spanish",model='sonnet',ignore_cache=True)
for r in results:
    print("Request ID#"+str(r[0]),r[1][1]," Bedrock response time:",r[1][3],"seconds.")

Request ID#1 Uno.  Bedrock response time: 0.42 seconds.
Request ID#2 Dos.  Bedrock response time: 0.51 seconds.
Request ID#3 Tres.  Bedrock response time: 0.65 seconds.
CPU times: user 52.8 ms, sys: 3.26 ms, total: 56 ms
Wall time: 660 ms


### Evaluation Function: evaluate_prompt()
Our final piece of setup is an evaluation function.  It is presented in an updated form here, but please [see this blog](https://medium.com/@flux07/prompt-evaluation-systematically-testing-and-improving-your-gen-ai-prompts-at-scale-784e54efe83d) for an explication of the basic idea.  First we have a function that get_results, which is a helper function just to take a prompt template an a bunch of inputs, and generate outputs.  These outputs can be fed into our main evaluation function, evaluate_results.  This uses an LLM as judge to provide an accuracy metric, as well as reasoning for each score.

In [157]:
from bs4 import BeautifulSoup as BS, re

def get_results(prompt_template,prompt_template_system, input_output,model,ignore_cache=False):
    '''
    Take a prompt template, and a set of inputs, and generate results for those inputs.
    prompt_template: the template to test, {{INPUT}} where input from the gold standard set should be slotted in.
    prompt_template_system: system prompt for the template to test
    input_output: an array of input/output pairs, human currated "correct" result examples.
    '''
    print ("Generating results to score...")
    prompts = []
    duplicate_check = {}
    for test_id,category,i,o in input_output:
        if i in duplicate_check:
            print ("Error!  The exact same prompt has been added to the gold standard set multiple times!")
            print ("Test ID %s and %s are the same prompt."%(test_id,duplicate_check[i]))
            raise()
        else:
            duplicate_check[i] = test_id
        prompt = prompt_template.replace("{{INPUT}}",i)
        prompts.append((test_id,prompt))
    
    #send all questions to Claude
    results_to_test = ask_claude_threaded(prompts, system=prompt_template_system,model=model,ignore_cache=ignore_cache)
    return results_to_test

def evaluate_results(input_output, results_to_test, test_prompt_template,test_prompt_template_system, model_used_for_results, answer_tag, threshhold = 80, print_out=False):
    '''
    Using LLM as Judge, score the prompt.
    input_output: an array of input/output pairs, human currated "correct" result examples.
    results_to_test: results from sending a bunch of prompts to ask_claude_threaded.
    test_prompt_template: prompt to use LLM as Judge, including {{OUTPUT}} for sloting in the output to test, and {{GOLD_OUTPUT}} for sloting in the gold standard result.
    test_prompt_template_system: system prompt for the LLM as judge
    model_used_for_results is the model that was used to generate these results, used to calculate cost.
    NOTE: Sonnet is always used to evaluate the results.
    answer_tag is the tag inside the prompt response that should be pulled out and considered to be the answer.  (this allows you to have things like 'reasoning' which does not get scored.)
    threshhold: from 0-100, how high score should be to be counted as passing.
    print_out: True to print a few statistics about the results.
    '''
    #build a dic for storing our results
    test_id2results = {}
    for test_id,category,i,o in input_output:
        test_id2results[test_id] = [category,i,o]
    
    prompts_to_score = []
    prompt_times = []
    prompt_cost = []
    for this_result in results_to_test:
        test_id = this_result[0]
        prompt = this_result[1][0]
        result = this_result[1][1]
        usage = this_result[1][2]
        time = this_result[1][3]
        prompt_times.append(time)
        prompt_cost.append(calculate_cost(usage, model_used_for_results))
        raw_result = result
        if "<"+answer_tag+">" in result:#strip the result down to the contents of the correct tag, if present.
            result = re.search("<"+answer_tag+">(.+?)</"+answer_tag+">",result.replace("\n","")).group(1)
        prompt_reason = ""
        if "<reason>" in raw_result:#strip the result down to the contents of the correct tag, if present.
            prompt_reason = re.search("<reason>(.+?)</reason>",raw_result.replace("\n","")).group(1)
        test_id2results[test_id].append(result)
        test_id2results[test_id].append(prompt_reason)
        test_prompt = test_prompt_template.replace("{{GOLD_OUTPUT}}",result).replace("{{OUTPUT}}",test_id2results[test_id][2])
        prompts_to_score.append((test_id,test_prompt))
    
    average_time = round(sum(prompt_times)/len(prompt_times),2)
    total_cost = sum(prompt_cost)
    print ("Average response time was %s seconds."%(average_time))
    print ("Total cost before scoring: $%s"%(round(total_cost,4)))
    print ("Scoring results...")
    #send all results to Claude
    scored_results = ask_claude_threaded(prompts_to_score, system=test_prompt_template_system)

    
    scores = []
    total_scored = 0
    total_passed = 0
    average_score = 0
    category_scores = {}
    for this_result in scored_results:
        test_id = this_result[0]
        prompt = this_result[1][0]
        response = this_result[1][1]
        usage = this_result[1][2]
        query_time = this_result[1][3]
        category,i,o,a,prompt_reason = test_id2results[test_id]

        soup = BS(response)
        score = soup.find('score').text
        reason = soup.find('reason').text
        average_score+=int(score)
        passed = True
        if int(score)<threshhold:
            passed = False
            
        #keep track for printing locally
        total_scored+=1
        if passed: total_passed+=1
        
        if category in category_scores:
            category_scores[category].append(int(score))
        else:
            category_scores[category] = [int(score)]
        
        scores.append([category,i,o,a,score,reason,passed,prompt_reason])
    average_score = round(average_score/total_scored,2)
    if print_out:
        print("Total inputs:",total_scored)
        print("Total Correct:",total_passed)
        print("Accuracy:",round(total_passed/total_scored,2)*100,"%")
        #print("Average Score:",average_score)
        
        categories = list(category_scores.keys())
        categories.sort()
        print ("Category Scores:")
        for cat in categories:
            print("  "+cat+":",round(sum(category_scores[cat])/len(category_scores[cat]),2))
    return scores

# 2) Examples of decomposition

## 2a) Task Based Decomposition

For this example, we consider a system that is intended to give recommendations for summer camps.  A parent can send in their child's interests, age, and required date for camp.  The system should then check for avalaible camps for that age and date, and create a recommendation based on the child's interest.  For this example, we are provided a list of 20 summer camps, and a list of 25 potential queries, along with the "correct" camp to reccomend for each query.  The correct camps were hand currated by a human, in order to insure they are correct.  Here is a diagram of the basic system:

![system](full_task.png)

And here is a decomposition of the system into three steps:

![Decomposed System](decomposed_task.png)

Here, we've broken the system into 3 steps:
1. Extract information, in this case the child's age, the date requests for camp, and the dates and age ranges that camps are offered for.
2. Find available camps by matching the child's age and date request to camps that accept that age on that date.
3. Based on the list of camps that are available for the child, recommend a camp based on their interests.

Note that LLM's are language models, not math models.  Something as simple as "Is 7 bigger than 3" should always be done by a program that can do math, and never left to an LLM.  By breaking down the task as shown above, we can do the 2.0 step of finding the correct camps in python, rather than depending on the LLM to calculate ranges.  

Skipping to the end, here are our results:

![results 1](results_example_1.png)

Not bad!  Using this method, we are able to rocket accuracy up to 100%, and cut costs as well.

Here is the list of camps:

In [214]:
camps = """<camp>
<name>Magical Unicorn Adventure Camp</name>
<date>June 12th - June 23rd</date>
<age_group>6 - 10 years old</age_group>
<description>Embark on a enchanting journey into the realm of unicorns! Children will learn about these mystical creatures, create unicorn-themed crafts, and even go on a quest to find a real unicorn in our enchanted forest.  This camp does not include any myths or other types of magic, and really only focuses on unicorns.</description>
</camp>

<camp>
<name>Rocket Science Camp</name>
<date>July 3rd - July 14th</date>
<age_group>10 - 14 years old</age_group>
<description>Blast off into the fascinating world of rocket science! Campers will learn about aerodynamics, propulsion systems, and the principles of space flight. They'll also get to design and launch their own model rockets.</description>
</camp>

<camp>
<name>Culinary Creations Camp</name>
<date>June 19th - June 30th</date>
<age_group>8 - 12 years old</age_group>
<description>Unleash your inner chef at this delectable camp! Young foodies will learn essential cooking skills, explore various cuisines, and create mouthwatering dishes to share with their families.</description>
</camp>

<camp>
<name>Wizard Academy</name>
<date>July 17th - July 28th</date>
<age_group>7 - 11 years old</age_group>
<description>Prepare to enter the magical realm of wizardry! Campers will learn the art of potion-making, wand-crafting, and even engage in friendly duels to hone their spellcasting abilities.  This is a pure fantasy based camp, perfect for kids that like reading fantasy books.</description>
</camp>

<camp>
<name>Coding Creators Camp</name>
<date>August 7th - August 18th</date>
<age_group>11 - 15 years old</age_group>
<description>Dive into the world of coding and technology! Campers will learn programming languages, develop their own games and apps, and explore the exciting possibilities of coding.</description>
</camp>

<camp>
<name>Wilderness Explorers Camp</name>
<date>June 26th - July 7th</date>
<age_group>9 - 13 years old</age_group>
<description>Embrace the great outdoors at this adventurous camp! Campers will learn survival skills, navigate through hiking trails, participate in team-building activities, and experience the thrill of camping under the stars.</description>
</camp>

<camp>
<name>Artistic Expressions Camp</name>
<date>July 31st - August 11th</date>
<age_group>6 - 10 years old</age_group>
<description>Unleash your creative side at this artistic haven! Young artists will explore various mediums such as painting, sculpture, and photography, while learning from professional artists and expressing their unique visions.</description>
</camp>

<camp>
<name>Robotics Innovators Camp</name>
<date>August 14th - August 25th</date>
<age_group>12 - 16 years old</age_group>
<description>Dive into the fascinating world of robotics! Campers will learn about robotics principles, programming, and mechatronics. They'll also get to design, build, and program their own robots to compete in exciting challenges.</description>
</camp>

<camp>
<name>Storytellers Camp</name>
<date>July 10th - July 21st</date>
<age_group>8 - 12 years old</age_group>
<description>Ignite your imagination and unleash your storytelling skills! Campers will explore various writing genres, create their own characters and worlds, and bring their stories to life through performance and illustration.</description>
</camp>

<camp>
<name>Eco-Warriors Camp</name>
<date>August 21st - September 1st</date>
<age_group>10 - 14 years old</age_group>
<description>Become an eco-warrior and learn about environmental conservation! Campers will participate in eco-friendly activities, learn about sustainable practices, and engage in hands-on projects to protect and preserve our planet.</description>
</camp>

<camp>
<name>Jedi Training Academy</name>
<date>June 12th - June 23rd</date>
<age_group>7 - 12 years old</age_group>
<description>Calling all aspiring Jedi Knights! In this camp, young Padawans will learn the ways of the Force, master lightsaber techniques, and embark on epic adventures to restore peace and balance to the galaxy.  This camp is all about sci-fi and Jedi, there is no magic or traditional fantasy elements.</description>
</camp>

<camp>
<name>Spy Kids Camp</name>
<date>July 3rd - July 14th</date>
<age_group>9 - 13 years old</age_group>
<description>Unlock your inner secret agent at this thrilling camp! Campers will learn espionage skills, code-breaking techniques, and even participate in exciting undercover missions to foil dastardly plots.</description>
</camp>

<camp>
<name>Circus Spectacular</name>
<date>June 19th - June 30th</date>
<age_group>6 - 10 years old</age_group>
<description>Step right up and join the circus! Young performers will learn acrobatics, juggling, clowning, and other circus arts from professional instructors, culminating in an amazing end-of-camp show.</description>
</camp>

<camp>
<name>Futuristic Inventors Camp</name>
<date>July 17th - July 28th</date>
<age_group>11 - 15 years old</age_group>
<description>Explore the cutting-edge world of future technology! Campers will learn about emerging technologies, design futuristic inventions, and even build working prototypes using 3D printing and other advanced tools.</description>
</camp>

<camp>
<name>Mythological Adventures</name>
<date>August 7th - August 18th</date>
<age_group>8 - 12 years old</age_group>
<description>Embark on a journey through the captivating realms of mythology! Campers will explore real ancient legends, create their own mythical creatures, and reenact epic battles between gods and monsters.  This is focused on the real global history of myths, and not the more fantastical elements of the stories.</description>
</camp>

<camp>
<name>Extreme Sports Camp</name>
<date>June 26th - July 7th</date>
<age_group>12 - 16 years old</age_group>
<description>Unleash your adventurous spirit at this adrenaline-pumping camp! Campers will have the opportunity to try extreme sports like rock climbing, skateboarding, BMX biking, and more, under the guidance of experienced instructors.</description>
</camp>

<camp>
<name>Musical Theatre Extravaganza</name>
<date>July 31st - August 11th</date>
<age_group>10 - 14 years old</age_group>
<description>Lights, camera, action! This camp is a musical theatre lover's dream. Campers will learn singing, acting, and dancing techniques, and work together to stage a spectacular Broadway-style production.</description>
</camp>

<camp>
<name>Entrepreneurial Sharks</name>
<date>August 14th - August 25th</date>
<age_group>13 - 17 years old</age_group>
<description>Dive into the world of business and entrepreneurship! Campers will learn essential skills like marketing, finance, and pitching ideas. They'll also have the opportunity to develop and present their own business plans to a panel of "shark" investors.</description>
</camp>

<camp>
<name>Forensic Detectives Camp</name>
<date>July 10th - July 21st</date>
<age_group>10 - 14 years old</age_group>
<description>Calling all aspiring crime scene investigators! At this camp, young detectives will learn about forensic science techniques, analyze evidence, and solve thrilling mock criminal cases.</description>
</camp>

<camp>
<name>World Explorers Camp</name>
<date>August 21st - September 1st</date>
<age_group>8 - 12 years old</age_group>
<description>Embark on a global adventure without leaving camp! Campers will explore different cultures, languages, customs, and cuisines from around the world, making it a truly enriching and educational experience.</description>
</camp>
"""

In [215]:
#orginize the info about each summer camp, for easy recall later.
from bs4 import BeautifulSoup
soup = BeautifulSoup(camps, "html.parser")
camp_data = soup.findAll("camp")
camp_dict = {}
camp_o = []
for camp in camp_data:
    date = camp.findAll("date")[0].text
    age = camp.findAll("age_group")[0].text
    name = camp.findAll("name")[0].text
    desc = camp.findAll("description")[0].text
    camp = {}
    camp["date"]=date
    camp["age"]=age
    camp["desc"]=desc
    camp_dict[name]=camp
    camp_o.append((date,age,name))
camp_o.sort()
for c in camp_o:
    pass#print(str(c)[1:-1])

Like every good Generative AI project, we will start by setting up our test cases so that we can measure the impact and quality of everything else we do.  (More information on the prompt evaluation used here can be found in [this blog](https://medium.com/@flux07/prompt-evaluation-systematically-testing-and-improving-your-gen-ai-prompts-at-scale-784e54efe83d))  Note that this notebook contains a significantly updates system for evaluation, so please only use that blog as a reference for the idea of prompt evaluation.

In [216]:
"""gold standard test cases.  The format is category, input, correct output.
Note that test cases should try to cover every major use case, as well as edge cases.
Categories we include here are:
  - basic use case.  Age and date range work, and the ask is clearly for a single camp.
  - second_pick.  Ask is for a camp that is out of range, but includes a second option that is in range.
  - age  child is too old or yound for camp
  - time The ask is for a time outside of when camps are offered.
  - indirect_time The desired date is asked for indirectly
  - indirect_age The disited age is asked for indirectly.
"""
test_cases = [
   [1,"basic","My 6-year-old daughter loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp on August 1st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?","Artistic Expressions Camp"],
   [2,"basic","My 6-year-old son is fascinated by the circus and loves to perform silly tricks and jokes. He has a natural flair for entertaining others. We're looking for a camp on June 20th that could nurture his interest in circus arts and allow him to explore his creative side. Could you recommend a suitable camp for him?","Circus Spectacular"],
   [3,"basic","My 7-year-old son is fascinated by magic and fantasy stories. He loves reading about wizards, casting spells, and going on magical adventures. We're looking for a camp on July 22nd that could fuel his imagination and allow him to explore the world of wizardry in a fun and engaging way.","Wizard Academy"],
   [4,"basic","My 8-year-old daughter is a budding artist who loves to express herself through various creative mediums like painting, drawing, and sculpting. We're looking for a summer camp on August 2nd that can nurture her artistic talents and provide her with an opportunity to explore different art forms under the guidance of experienced instructors. Can you recommend a suitable camp for her?","Artistic Expressions Camp"],
   [5,"basic","My 10-year-old daughter is absolutely enchanted by the world of unicorns and all things magical. She loves crafting, exploring nature, and immersing herself in fantastical stories. I'm looking for a camp on June 20th that would cater to her interests and allow her to unleash her imagination in a whimsical setting.","Magical Unicorn Adventure Camp"],
   [6,"second_pick","My 10-year-old son is fascinated by robots and loves to program.  I really need somewhere he can learn about innovating with robots. He also reads about mythical creatures when he’s bored. We're looking for a summer camp on August 9th that could build his ability to innovate. Could you recommend a suitable camp for him?","Mythological Adventures"],
   [7,"second_pick","My 11-year-old daughter is passionate about singing, acting, and dancing. Her dream is to perform in the circus!  She loves clowns so much.  We are looking for a camp during the week of August 11th that would allow her to explore her interests further and potentially participate in a production. Can you recommend a suitable camp for her?","Musical Theatre Extravaganza"],
   [8,"second_pick","My 16-year-old son is interested in attending a summer camp on August 19th. He has a keen interest in extreme sports, he loves soccer and football.  When not having fun with extreem sports, he's always coming up with ideas for new products or services. Could you recommend a camp that would allow him to explore these interests and develop his skills?","Entrepreneurial Sharks"],
   [9,"second_pick","My 17-year-old son is fascinated by technology and has a keen interest in robotics. He once built a whole robot from a box of spare parts while in a cave in the desert!  He'll be available on August 20th, and we're looking for a camp that would allow him to explore his passion for building and programming robots. When the power is out, he also loves making up wth new ideas for how to sell robots.  Could you recommend a suitable camp that aligns with his interests and the desired date?","Entrepreneurial Sharks"],
   [10,"age","My 5-year-old daughter is fascinated by unicorns and all things magical. She has a vivid imagination and loves creating her own stories and adventures. We're looking for a camp on July 4th that caters to her interests and allows her to explore her creativity in a fun and engaging way.","Sorry, we do not have any summer camps that match your request."],
   [11,"age","My 5-year-old daughter is very imaginative and loves anything related to magic and fantasy. We're looking for a camp on August 11th that could spark her creativity and allow her to explore her interests in a fun and engaging way.","Sorry, we do not have any summer camps that match your request."],
   [12,"age","My 18-year-old son is interested in attending a summer camp on June 10th. He has a keen interest in entrepreneurship, business, and pitching ideas to potential investors. Could you recommend a camp that aligns with his interests and age group?","Sorry, we do not have any summer camps that match your request."],
   [13,"age","My 18-year-old son is interested in starting new businesses. He would love to attend a camp on August 14th that aligns with his interests and allows him to explore cutting-edge concepts while also having fun.","Sorry, we do not have any summer camps that match your request."],
   [14,"time","My 10-year-old son is really interested in science and technology. He loves building things and learning about how things work. We're looking for a camp on June 10th that could nurture his curiosity and allow him to explore his interests in a fun and engaging way.","Sorry, we do not have any summer camps that match your request."],
   [15,"time","My 17-year-old son is really interested in starting new businesses. He loves building things and learning about how things work. We're looking for a camp on July 20th that could nurture his curiosity and allow him to explore his interests in a fun and engaging way.","Sorry, we do not have any summer camps that match your request."],
   [16,"time","My 7-year-old son is available on August 12th, and he's absolutely fascinated by anything related to magic, fantasy, and mythical creatures. Could you recommend a camp that aligns with his interests and the available dates?","Sorry, we do not have any summer camps that match your request."],
   [17,"time","My 6-year-old son is really interested in fantasy and magic. He loves stories about wizards, unicorns, and mythical creatures. We're looking for a camp on July 20th that could spark his imagination and allow him to explore these interests in a fun and engaging way.","Sorry, we do not have any summer camps that match your request."],
   [18,"indirect_time","My 6-year-old daughter loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp two days before August 3st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?","Artistic Expressions Camp"],
   [19,"indirect_time","My 6-year-old son is fascinated by the circus and loves to perform silly tricks and jokes. He has a natural flair for entertaining others. We're looking for a camp a week after June 13th that could nurture his interest in circus arts and allow him to explore his creative side. Could you recommend a suitable camp for him?","Circus Spectacular"],
   [20,"indirect_time","My 7-year-old son is fascinated by magic and fantasy stories. He loves reading about wizards, casting spells, and going on magical adventures. We're looking for a camp a month after June 22nd that could fuel his imagination and allow him to explore the world of wizardry in a fun and engaging way.","Wizard Academy"],
   [21,"indirect_time","My 8-year-old daughter is a budding artist who loves to express herself through various creative mediums like painting, drawing, and sculpting. We're looking for a summer camp the day before August 3rd that can nurture her artistic talents and provide her with an opportunity to explore different art forms under the guidance of experienced instructors. Can you recommend a suitable camp for her?","Artistic Expressions Camp"],
   [22,"indirect_age","My 5-year-old daughter has a birthday May 3rd this year, and loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp on August 1st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?","Artistic Expressions Camp"],
   [23,"indirect_age","My older son is 7, and my younger is 6.  I need a camp for the younger one, he is fascinated by the circus and loves to perform silly tricks and jokes. He has a natural flair for entertaining others. We're looking for a camp on June 20th that could nurture his interest in circus arts and allow him to explore his creative side. Could you recommend a suitable camp for him?","Circus Spectacular"],
   [24,"indirect_age","My 6-year-old son is going the have a wizard themed birthday tomorow.  He is fascinated by magic and fantasy stories. He loves reading about wizards, casting spells, and going on magical adventures. We're looking for a camp on July 22nd that could fuel his imagination and allow him to explore the world of wizardry in a fun and engaging way.","Wizard Academy"],
   [25,"indirect_age","I have two boys, 5 and 10, and a daughter who is 8.  My daughter is a budding artist who loves to express herself through various creative mediums like painting, drawing, and sculpting. We're looking for a summer camp on August 2nd that can nurture her artistic talents and provide her with an opportunity to explore different art forms under the guidance of experienced instructors. Can you recommend a suitable camp for her?","Artistic Expressions Camp"]
]
print ("%s test cases created."%len(test_cases))

25 test cases created.


Next, we create the prompt that will attempt to solve the entire task in a single step.

In [217]:
prompt_template_system = """You are a summer camp advisor, in charge of placing kids into summer camps based on their interest.
Here is the list of summer camps that are avalaible this summer:
{{CAMPS}}
It is critical that kids are only placed in camps approved for their age.
It is critical that kids only join camps that are offered on the day they are requesting.
Every camp offers single day drop in any day in its date range.
If no camp can be found for the age and date requested, please say "Sorry, we do not have any summer camps that match your request."
"""
prompt_template_system = prompt_template_system.replace("{{CAMPS}}",camps)
prompt_template = """Here is a request from a parent:
<text>
{{INPUT}}
</text>
Which camp do you recommend?  Put "Sorry, we do not have any summer camps that match your request." or the camp name in 'camp' XML tags, and your reasoning in 'reason' tags.
"""

In [291]:
#Test that the prompt is working
text = "My 6-year-old daughter loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp on August 1st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?"
result = ask_claude(prompt_template.replace("{{INPUT}}",text),system=prompt_template_system)
print (result[1])

<camp>Artistic Expressions Camp</camp>
<reason>The Artistic Expressions Camp, scheduled from July 31st to August 11th, is a perfect fit for your 6-year-old daughter's interests and age. This camp is designed for children aged 6 to 10 years old and allows them to explore various artistic mediums such as painting, sculpture, and photography. With professional artists as instructors, your daughter will have the opportunity to unleash her creativity and express her unique artistic vision. Since August 1st falls within the camp's date range, she can attend as a single-day drop-in participant.</reason>


In order to measure accuracy, we will be using an LLM to judge.  In this case, we have a straight forward answer matching problem - matching the expected camp name with the camp suggested by running the prompt.  In particular, this is a pass / fail, so we create a judge prompt that will give a score of 100 to matches, and 0 to wrong answers.

In [219]:
test_prompt_template_system = """You are a detail oriented teacher.  

You are grading an exam, looking at a correct answer and a student submitted answer.  Your goal is to score the student answer based on how close it is to the correct answer.

This is a pass/fail test.  If the two answers are basically the same, the score should be 100.
Minor things like punctuation, capitilization, or spelling should not impact the score.
If the two answers are different, then the score should be 0.
Please you your score in a 'score' XML tag, and any reasoning in a 'reason' XML tag.
"""
test_prompt_template = """
Please score this submission.
Here is the correct answer:
<correct_answer>{{GOLD_OUTPUT}}</correct_answer>
Here is the student's answer:
<student_answer>{{OUTPUT}}</student_answer>"""

In [292]:
#let's run the tests!  We run with both Haiku and Sonnet so that we can compare results using different models.  
answer_tag = 'camp'
print ("Sonnet:")
results_to_test = get_results(prompt_template,prompt_template_system, test_cases,model="sonnet", ignore_cache = False)
scores = evaluate_results(test_cases, results_to_test, test_prompt_template,test_prompt_template_system, "sonnet", answer_tag, threshhold = 80, print_out=True)
print ("Haiku:")
results_to_test = get_results(prompt_template,prompt_template_system, test_cases, model="haiku")
scores_haiku = evaluate_results(test_cases, results_to_test, test_prompt_template,test_prompt_template_system, "haiku", answer_tag, threshhold = 80, print_out=True)

Sonnet:
Generating results to score...
Average response time was 4.38 seconds.
Total cost before scoring: $0.2387
Scoring results...
Total inputs: 25
Total Correct: 15
Accuracy: 60.0 %
Category Scores:
  age: 25.0
  basic: 100.0
  indirect_age: 100.0
  indirect_time: 100.0
  second_pick: 25.0
  time: 0.0
Haiku:
Generating results to score...
Average response time was 1.84 seconds.
Total cost before scoring: $0.0199
Scoring results...
Total inputs: 25
Total Correct: 16
Accuracy: 64.0 %
Category Scores:
  age: 75.0
  basic: 100.0
  indirect_age: 75.0
  indirect_time: 100.0
  second_pick: 25.0
  time: 0.0


#### Interesting!  Our accuracy averaged around 60%.  Notice that both Haiku and Sonnet generated about the same accuracy for this task, sometimes with Haiku even doing slightly better!  This underscores the importance of always testing and evaluating for the correct size model for your task.  It also shows that this accuracy metrics are estimates.

Let's take a quick peek at where the prompt got things wrong, which can help give ideas about what might be imporved next time.

In [293]:
already_done = []
for category, question,correct_answer,prompt_answer,score,reason,passed,prompt_reason in scores:
    if int(score)>=50:continue
    if category in already_done:continue #look at one example per category
    already_done.append(category)
    print ()
    print ("Category:",category,"Score:",score)
    print ("Question:",question)
    print ("Gold Ans:",correct_answer)
    print ("LLM  Ans:",prompt_answer.strip())
    print ("LLM's reasoning:\n"+prompt_reason)
    print ("_________________")
print ("")


Category: second_pick Score: 0
Question: My 10-year-old son is fascinated by robots and loves to program.  I really need somewhere he can learn about innovating with robots. He also reads about mythical creatures when he’s bored. We're looking for a summer camp on August 9th that could build his ability to innovate. Could you recommend a suitable camp for him?
Gold Ans: Mythological Adventures
LLM  Ans: Robotics Innovators Camp
LLM's reasoning:
The Robotics Innovators Camp is a perfect fit for your 10-year-old son's interests in robots and programming. The camp runs from August 14th to August 25th, which includes August 9th, the date you requested. According to the description, campers will learn about robotics principles, programming, and mechatronics, as well as design, build, and program their own robots. This camp aligns with your son's fascination with robots and his desire to learn about innovation. Additionally, the age group of 12-16 years old is suitable for your 10-year-old 

## Now, let's decompose this task, and increase the quality of our results.
Original task:

  - Task 1: Read in parent's request -> produce a recommendation

Decomposed task:

  - Task 1: Read in parent's request -> extract required age and date
  - prep task:  One time, extract the date and age requirments for each camp.  We reuse these extracted dates for all requests.
  - Task 2: For a given age and date -> extract a list of matching camps
  - Task 3: From a list of matching camps -> produce a recommendation.

Let's create 2 seperate prompts, one for Step 1, and another for Step 3.  Step 2 is a simple range compare, so we can use Python for that.  We will also create test cases for each step, so that we can monitor the accuracy of each step.


#### Step 1: Extract the request age and date information

In [222]:
test_cases_step_1 = [
   [1,"basic","My 6-year-old daughter loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp on August 1st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?","6,20250801"],
   [2,"basic","My 6-year-old son is fascinated by the circus and loves to perform silly tricks and jokes. He has a natural flair for entertaining others. We're looking for a camp on June 20th that could nurture his interest in circus arts and allow him to explore his creative side. Could you recommend a suitable camp for him?","6,20250620"],
   [3,"basic","My 7-year-old son is fascinated by magic and fantasy stories. He loves reading about wizards, casting spells, and going on magical adventures. We're looking for a camp on July 22nd that could fuel his imagination and allow him to explore the world of wizardry in a fun and engaging way.","7,20250722"],
   [4,"basic","My 8-year-old daughter is a budding artist who loves to express herself through various creative mediums like painting, drawing, and sculpting. We're looking for a summer camp on August 2nd that can nurture her artistic talents and provide her with an opportunity to explore different art forms under the guidance of experienced instructors. Can you recommend a suitable camp for her?","8,20250802"],
   [5,"basic","My 10-year-old daughter is absolutely enchanted by the world of unicorns and all things magical. She loves crafting, exploring nature, and immersing herself in fantastical stories. I'm looking for a camp on June 20th that would cater to her interests and allow her to unleash her imagination in a whimsical setting.","10,20250620"],
   [6,"second_pick","My 10-year-old son is fascinated by robots and loves to program.  I really need somewhere he can learn about innovating with robots. He also reads about mythical creatures when he’s bored. We're looking for a summer camp on August 9th that could build his ability to innovate. Could you recommend a suitable camp for him?","10,20250809"],
   [7,"second_pick","My 11-year-old daughter is passionate about singing, acting, and dancing. Her dream is to perform in the circus!  She loves clowns so much.  We are looking for a camp during the week of August 11th that would allow her to explore her interests further and potentially participate in a production. Can you recommend a suitable camp for her?","11,20250811"],
   [8,"second_pick","My 16-year-old son is interested in attending a summer camp on August 19th. He has a keen interest in extreme sports, he loves soccer and football.  When not having fun with extreem sports, he's always coming up with ideas for new products or services. Could you recommend a camp that would allow him to explore these interests and develop his skills?","16,20250819"],
   [9,"second_pick","My 17-year-old son is fascinated by technology and has a keen interest in robotics. He once built a whole robot from a box of spare parts while in a cave in the desert!  He'll be available on August 20th, and we're looking for a camp that would allow him to explore his passion for building and programming robots. When the power is out, he also loves making up wth new ideas for how to sell robots.  Could you recommend a suitable camp that aligns with his interests and the desired date?","17,20250820"],
   [10,"age","My 5-year-old daughter is fascinated by unicorns and all things magical. She has a vivid imagination and loves creating her own stories and adventures. We're looking for a camp on July 4th that caters to her interests and allows her to explore her creativity in a fun and engaging way.","5,20250704"],
   [11,"age","My 5-year-old daughter is very imaginative and loves anything related to magic and fantasy. We're looking for a camp on August 11th that could spark her creativity and allow her to explore her interests in a fun and engaging way.","5,20250811"],
   [12,"age","My 18-year-old son is interested in attending a summer camp on June 10th. He has a keen interest in entrepreneurship, business, and pitching ideas to potential investors. Could you recommend a camp that aligns with his interests and age group?","18,20250610"],
   [13,"age","My 18-year-old son is interested in starting new businesses. He would love to attend a camp on August 14th that aligns with his interests and allows him to explore cutting-edge concepts while also having fun.","18,20250814"],
   [14,"time","My 10-year-old son is really interested in science and technology. He loves building things and learning about how things work. We're looking for a camp on June 10th that could nurture his curiosity and allow him to explore his interests in a fun and engaging way.","10,20250610"],
   [15,"time","My 17-year-old son is really interested in starting new businesses. He loves building things and learning about how things work. We're looking for a camp on July 20th that could nurture his curiosity and allow him to explore his interests in a fun and engaging way.","17,20250720"],
   [16,"time","My 7-year-old son is available on August 12th, and he's absolutely fascinated by anything related to magic, fantasy, and mythical creatures. Could you recommend a camp that aligns with his interests and the available dates?","7,20250812"],
   [17,"time","My 6-year-old son is really interested in fantasy and magic. He loves stories about wizards, unicorns, and mythical creatures. We're looking for a camp on July 20th that could spark his imagination and allow him to explore these interests in a fun and engaging way.","6,20250720"],
   [18,"indirect_time","My 6-year-old daughter loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp two days before August 3st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?","6,20250801"],
   [19,"indirect_time","My 6-year-old son is fascinated by the circus and loves to perform silly tricks and jokes. He has a natural flair for entertaining others. We're looking for a camp a week after June 13th that could nurture his interest in circus arts and allow him to explore his creative side. Could you recommend a suitable camp for him?","6,20250620"],
   [20,"indirect_time","My 7-year-old son is fascinated by magic and fantasy stories. He loves reading about wizards, casting spells, and going on magical adventures. We're looking for a camp a month after June 22nd that could fuel his imagination and allow him to explore the world of wizardry in a fun and engaging way.","7,20250722"],
   [21,"indirect_time","My 8-year-old daughter is a budding artist who loves to express herself through various creative mediums like painting, drawing, and sculpting. We're looking for a summer camp the day before August 3rd that can nurture her artistic talents and provide her with an opportunity to explore different art forms under the guidance of experienced instructors. Can you recommend a suitable camp for her?","8,20250802"],
   [22,"indirect_age","My 5-year-old daughter has a birthday May 3rd this year, and loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp on August 1st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?","6,20250801"],
   [23,"indirect_age","My older son is 7, and my younger is 6.  I need a camp for the younger one, he is fascinated by the circus and loves to perform silly tricks and jokes. He has a natural flair for entertaining others. We're looking for a camp on June 20th that could nurture his interest in circus arts and allow him to explore his creative side. Could you recommend a suitable camp for him?","6,20250620"],
   [24,"indirect_age","My 6-year-old son is going the have a wizard themed birthday tomorow.  He is fascinated by magic and fantasy stories. He loves reading about wizards, casting spells, and going on magical adventures. We're looking for a camp on July 22nd that could fuel his imagination and allow him to explore the world of wizardry in a fun and engaging way.","7,20250722"],
   [25,"indirect_age","I have two boys, 5 and 10, and a daughter who is 8.  My daughter is a budding artist who loves to express herself through various creative mediums like painting, drawing, and sculpting. We're looking for a summer camp on August 2nd that can nurture her artistic talents and provide her with an opportunity to explore different art forms under the guidance of experienced instructors. Can you recommend a suitable camp for her?","8,20250802"]
]

In [223]:
prompt_template_step_1_system = """You are detail oriented data extraction expert.
Your job is to examine a short message which is a request from a parent for summer camp advice, and extract the age of the child and desired date for camp.
First, extract the exact age mentioned in the request, and check to see if a birthday is mentioned.  Write your answer in a 'temp' XML tag, in the format (age,True/False birthday mentioned.)
Only when a birthday is stated, and the birthday is before the requested date of camp, add one to the value in 'temp' and consider that to be the new age.
Age is of critical importance to the process, so you must never assume or guess when a birthday is.  Instead, you must always use the exact age given by the parent, except when a pre-camp birthday is explictly mentioned.
Next, inside an 'answer' XML tag, write the requested age and date as "age,date"
Finally, include your reasoning in 'reason' tags.
The date should be in YYYYMMDD format, and the current year is 2025.
"""
prompt_template_step_1 = """Here is a message for data extraction:
<text>
{{INPUT}}
</text>
"""

In [224]:
text = "My 6-year-old daughter loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp around August 1st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?"
result = ask_claude(prompt_template_step_1.replace("{{INPUT}}",text),system=prompt_template_step_1_system)
print (result[1])

<temp>6,False</temp>
<answer>6,20250801</answer>
<reason>
The parent explicitly states that their child is 6 years old, with no mention of an upcoming birthday before the requested camp date of around August 1st. Therefore, the age to consider for camp recommendations is 6. The requested date of around August 1st is interpreted as August 1, 2025 in the YYYYMMDD format.
</reason>


In [286]:
#we can use the same testing prompts as we did before.
answer_tag = 'answer'
print ("Sonnet:")
results_to_test = get_results(prompt_template_step_1,prompt_template_step_1_system, test_cases_step_1,model="sonnet", ignore_cache = False)
scores = evaluate_results(test_cases_step_1,results_to_test, test_prompt_template,test_prompt_template_system, "sonnet", answer_tag, threshhold = 80, print_out=True)
print ("Haiku:")
results_to_test = get_results(prompt_template_step_1,prompt_template_step_1_system, test_cases_step_1, model="haiku",ignore_cache=False)
scores_haiku = evaluate_results(test_cases_step_1, results_to_test, test_prompt_template,test_prompt_template_system, "haiku", answer_tag, threshhold = 80, print_out=True)

Sonnet:
Generating results to score...
Average response time was 2.58 seconds.
Total cost before scoring: $0.0655
Scoring results...
Total inputs: 25
Total Correct: 25
Accuracy: 100.0 %
Category Scores:
  age: 100.0
  basic: 100.0
  indirect_age: 100.0
  indirect_time: 100.0
  second_pick: 100.0
  time: 100.0
Haiku:
Generating results to score...
Average response time was 1.41 seconds.
Total cost before scoring: $0.0047
Scoring results...
Total inputs: 25
Total Correct: 21
Accuracy: 84.0 %
Category Scores:
  age: 100.0
  basic: 100.0
  indirect_age: 75.0
  indirect_time: 25.0
  second_pick: 100.0
  time: 100.0


Accuracy on this first step looks pretty good - Sonnet gave us 100%, or if we don't need it to be quite that high, we could use Haiku and get most of the way there.

In [226]:
#pack up these answers for use in the next step.
question2ID = {}
for test_id,cat,i,o in test_cases_step_1:
    question2ID[i]=test_id
testID2age_date = {}
for category, question,correct_answer,prompt_answer,score,reason,passed,prompt_reason in scores:
    testID2age_date[question2ID[question]] = prompt_answer

In [227]:
#take a peak at one of the wrong answers in each category, so that we can see the LLM's reasoning.
already_done = []
for category, question,correct_answer,prompt_answer,score,reason,passed,prompt_reason in scores_haiku:
    if int(score)>=50:continue
    if category in already_done:continue #look at one example per category
    already_done.append(category)
    print ()
    print ("Category:",category,"Score:",score)
    print ("Question:",question)
    print ("Gold Ans:",correct_answer)
    print ("LLM  Ans:",prompt_answer.strip())
    print ("LLM's reasoning:\n"+prompt_reason)
    print ("_________________")
print ("")


Category: basic Score: 0
Question: My 6-year-old son is fascinated by the circus and loves to perform silly tricks and jokes. He has a natural flair for entertaining others. We're looking for a camp on June 20th that could nurture his interest in circus arts and allow him to explore his creative side. Could you recommend a suitable camp for him?
Gold Ans: 6,20250620
LLM  Ans: 7,20250620
LLM's reasoning:
The message states that the child is 6 years old, and does not mention a birthday. Therefore, the age is 6. The requested date for the camp is June 20, 2025.
_________________

Category: indirect_time Score: 0
Question: My 6-year-old daughter loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp two days before August 3st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?
Gold Ans: 6,20250801
LLM  Ans: 7,2025080

#### Step 2: Extract and compare the camp date and age information
Start with the gold standard set:

In [228]:
test_cases_step_2 = [
   [1,"basic","""<camp>
<name>Magical Unicorn Adventure Camp</name>
<date>June 12th - June 23rd</date>
<age_group>6 - 10 years old</age_group>
<description>Embark on a enchanting journey into the realm of unicorns! Children will learn about these mystical creatures, create unicorn-themed crafts, and even go on a quest to find a real unicorn in our enchanted forest.  This camp does not include any myths or other types of magic, and really only focuses on unicorns.</description>
</camp>""","[6,10,20250612,20250623]"],
   [2,"basic","""<camp>
<name>Rocket Science Camp</name>
<date>July 3rd - July 14th</date>
<age_group>10 - 14 years old</age_group>
<description>Blast off into the fascinating world of rocket science! Campers will learn about aerodynamics, propulsion systems, and the principles of space flight. They'll also get to design and launch their own model rockets.</description>
</camp>""","[10,14,20250703,20250714]"],
   [3,"basic","""<camp>
<name>Culinary Creations Camp</name>
<date>June 19th - June 30th</date>
<age_group>8 - 12 years old</age_group>
<description>Unleash your inner chef at this delectable camp! Young foodies will learn essential cooking skills, explore various cuisines, and create mouthwatering dishes to share with their families.</description>
</camp>""","[8,12,20250619,20250630]"],
   [4,"basic","""<camp>
<name>Wizard Academy</name>
<date>July 17th - July 28th</date>
<age_group>7 - 11 years old</age_group>
<description>Prepare to enter the magical realm of wizardry! Campers will learn the art of potion-making, wand-crafting, and even engage in friendly duels to hone their spellcasting abilities.  This is a pure fantasy based camp, perfect for kids that like reading fantasy books.</description>
</camp>""","[7,11,20250717,20250728]"],
   [5,"basic","""<camp>
<name>Coding Creators Camp</name>
<date>August 7th - August 18th</date>
<age_group>11 - 15 years old</age_group>
<description>Dive into the world of coding and technology! Campers will learn programming languages, develop their own games and apps, and explore the exciting possibilities of coding.</description>
</camp>""","[11,15,20250807,20250818]"],
   [6,"basic","""<camp>
<name>Wilderness Explorers Camp</name>
<date>June 26th - July 7th</date>
<age_group>9 - 13 years old</age_group>
<description>Embrace the great outdoors at this adventurous camp! Campers will learn survival skills, navigate through hiking trails, participate in team-building activities, and experience the thrill of camping under the stars.</description>
</camp>""","[9,13,20250626,20250707]"],
   [7,"basic","""<camp>
<name>Artistic Expressions Camp</name>
<date>July 31st - August 11th</date>
<age_group>6 - 10 years old</age_group>
<description>Unleash your creative side at this artistic haven! Young artists will explore various mediums such as painting, sculpture, and photography, while learning from professional artists and expressing their unique visions.</description>
</camp>""","[6,10,20250731,20250811]"],
   [8,"basic","""<camp>
<name>Robotics Innovators Camp</name>
<date>August 14th - August 25th</date>
<age_group>12 - 16 years old</age_group>
<description>Dive into the fascinating world of robotics! Campers will learn about robotics principles, programming, and mechatronics. They'll also get to design, build, and program their own robots to compete in exciting challenges.</description>
</camp>""","[12,16,20250814,20250825]"],
   [9,"basic","""<camp>
<name>Storytellers Camp</name>
<date>July 10th - July 21st</date>
<age_group>8 - 12 years old</age_group>
<description>Ignite your imagination and unleash your storytelling skills! Campers will explore various writing genres, create their own characters and worlds, and bring their stories to life through performance and illustration.</description>
</camp>""","[8,12,20250710,20250721]"],
   [10,"basic","""<camp>
<name>Eco-Warriors Camp</name>
<date>August 21st - September 1st</date>
<age_group>10 - 14 years old</age_group>
<description>Become an eco-warrior and learn about environmental conservation! Campers will participate in eco-friendly activities, learn about sustainable practices, and engage in hands-on projects to protect and preserve our planet.</description>
</camp>""","[10,14,20250821,20250901]"],
   [11,"basic","""<camp>
<name>Jedi Training Academy</name>
<date>June 12th - June 23rd</date>
<age_group>7 - 12 years old</age_group>
<description>Calling all aspiring Jedi Knights! In this camp, young Padawans will learn the ways of the Force, master lightsaber techniques, and embark on epic adventures to restore peace and balance to the galaxy.  This camp is all about sci-fi and Jedi, there is no magic or traditional fantasy elements.</description>
</camp>""","[7,12,20250612,20250623]"],
   [12,"basic","""<camp>
<name>Spy Kids Camp</name>
<date>July 3rd - July 14th</date>
<age_group>9 - 13 years old</age_group>
<description>Unlock your inner secret agent at this thrilling camp! Campers will learn espionage skills, code-breaking techniques, and even participate in exciting undercover missions to foil dastardly plots.</description>
</camp>""","[9,13,20250703,20250714]"],
   [13,"basic","""<camp>
<name>Circus Spectacular</name>
<date>June 19th - June 30th</date>
<age_group>6 - 10 years old</age_group>
<description>Step right up and join the circus! Young performers will learn acrobatics, juggling, clowning, and other circus arts from professional instructors, culminating in an amazing end-of-camp show.</description>
</camp>""","[6,10,20250619,20250630]"],
   [14,"basic","""<camp>
<name>Futuristic Inventors Camp</name>
<date>July 17th - July 28th</date>
<age_group>11 - 15 years old</age_group>
<description>Explore the cutting-edge world of future technology! Campers will learn about emerging technologies, design futuristic inventions, and even build working prototypes using 3D printing and other advanced tools.</description>
</camp>""","[11,15,20250717,20250728]"],
   [15,"basic","""<camp>
<name>Mythological Adventures</name>
<date>August 7th - August 18th</date>
<age_group>8 - 12 years old</age_group>
<description>Embark on a journey through the captivating realms of mythology! Campers will explore real ancient legends, create their own mythical creatures, and reenact epic battles between gods and monsters.  This is focused on the real global history of myths, and not the more fantastical elements of the stories.</description>
</camp>""","[8,12,20250807,20250818]"],
   [16,"basic","""<camp>
<name>Extreme Sports Camp</name>
<date>June 26th - July 7th</date>
<age_group>12 - 16 years old</age_group>
<description>Unleash your adventurous spirit at this adrenaline-pumping camp! Campers will have the opportunity to try extreme sports like rock climbing, skateboarding, BMX biking, and more, under the guidance of experienced instructors.</description>
</camp>""","[12,16,20250626,20250707]"],
   [17,"basic","""<camp>
<name>Musical Theatre Extravaganza</name>
<date>July 31st - August 11th</date>
<age_group>10 - 14 years old</age_group>
<description>Lights, camera, action! This camp is a musical theatre lover's dream. Campers will learn singing, acting, and dancing techniques, and work together to stage a spectacular Broadway-style production.</description>
</camp>""","[10,14,20250731,20250811]"],
   [18,"basic","""<camp>
<name>Entrepreneurial Sharks</name>
<date>August 14th - August 25th</date>
<age_group>13 - 17 years old</age_group>
<description>Dive into the world of business and entrepreneurship! Campers will learn essential skills like marketing, finance, and pitching ideas. They'll also have the opportunity to develop and present their own business plans to a panel of "shark" investors.</description>
</camp>""","[13,17,20250814,20250825]"],
   [19,"basic","""<camp>
<name>Forensic Detectives Camp</name>
<date>July 10th - July 21st</date>
<age_group>10 - 14 years old</age_group>
<description>Calling all aspiring crime scene investigators! At this camp, young detectives will learn about forensic science techniques, analyze evidence, and solve thrilling mock criminal cases.</description>
</camp>""","[10,14,20250710,20250721]"],
   [20,"basic","""<camp>
<name>World Explorers Camp</name>
<date>August 21st - September 1st</date>
<age_group>8 - 12 years old</age_group>
<description>Embark on a global adventure without leaving camp! Campers will explore different cultures, languages, customs, and cuisines from around the world, making it a truly enriching and educational experience.</description>
</camp>
""","[8,12,20250821,20250901]"]
]

#store camps in a dict for later.
campID2camp = {}
for campID, cat,camp,out in test_cases_step_2:
    campID2camp[campID]=camp

In [229]:
prompt_template_step_2_system = """You are a data extraction expert.
You will be given information about a summer camp.
Your task is to find information about the targeted age group for the camp, and the start and end dates.
Your answer should be a list inside an 'answer' XML tag.
The list should have the format [youngest age, oldest age, start date, end date].
All dates should be in YYYYMMDD format, and the current year is 2025.
"""
prompt_template_step_2 = """Here is infomation about a summer camp:
<text>
{{INPUT}}
</text>
"""

In [230]:
text = """<camp><name>World Explorers Camp</name>
<date>August 21st - September 1st</date>
<age_group>8 - 12 years old</age_group>
<description>Embark on a global adventure without leaving camp! Campers will explore different cultures, languages, customs, and cuisines from around the world, making it a truly enriching and educational experience.</description>
</camp>"""
result = ask_claude(prompt_template_step_2.replace("{{INPUT}}",text),system=prompt_template_step_2_system)
print (result[1])

<answer>[8, 12, 20250821, 20250901]</answer>


In [231]:
#we can use the same testing prompt as before.
answer_tag = 'answer'
print ("Sonnet:")
results_to_test = get_results(prompt_template_step_2,prompt_template_step_2_system, test_cases_step_2,model="sonnet", ignore_cache = False)
scores = evaluate_results(test_cases_step_2, results_to_test, test_prompt_template,test_prompt_template_system, "sonnet", answer_tag, threshhold = 80, print_out=True)
print ("Haiku:")
results_to_test = get_results(prompt_template_step_2,prompt_template_step_2_system, test_cases_step_2, model="haiku",ignore_cache= False)
scores_haiku = evaluate_results(test_cases_step_2, results_to_test, test_prompt_template,test_prompt_template_system, "haiku", answer_tag, threshhold = 80, print_out=True)

Sonnet:
Generating results to score...
Average response time was 1.13 seconds.
Total cost before scoring: $0.0212
Scoring results...
Total inputs: 20
Total Correct: 20
Accuracy: 100.0 %
Category Scores:
  basic: 100.0
Haiku:
Generating results to score...
Average response time was 0.61 seconds.
Total cost before scoring: $0.0019
Scoring results...
Total inputs: 20
Total Correct: 20
Accuracy: 100.0 %
Category Scores:
  basic: 100.0


This task was pretty easy, and even Haiku gave us 100% accuracy.  No need to use larger models here!

In [232]:
#pack up these answers for use in the next step.
camp2campID = {}
for test_id,cat,i,o in test_cases_step_2:
    camp2campID[i]=test_id
campID2age_date = {}
for category, question,correct_answer,prompt_answer,score,reason,passed,prompt_reason in scores_haiku:
    campID2age_date[camp2campID[question]] = prompt_answer

#### Step 3: Generate the final recommendation
For this step, we'll do the exact same thing we did in the original pre-decomposed prompt, except we will use the information from steps 1 and 2 to limit the number of camp options passed to the LLM.

#### First, we move the list of camps from the system prompt to the user prompt, because the list is different for each request.

In [241]:
prompt_template_decomposed_system = """You are a summer camp advisor, in charge of placing kids into summer camps based on their interest.
If no camp can be found for the age and date requested, please say "Sorry, we do not have any summer camps that match your request."
You will be given a list of summer camps that a director has already approved as possible options based on age and date.
You may safely ignore any age and date information, and instead focus on reccomending the camp that is best aligned with the child's interest.
"""
prompt_template_decomposed = """
Here is the list of summer camps that are avalaible for this request:
<camps>
{{CAMPS}}
</camps>
Here is a request from a parent:
<text>
{{INPUT}}
</text>
Which camp do you recommend?  Put "Sorry, we do not have any summer camps that match your request." or the camp name in 'camp' XML tags, and your reasoning in 'reason' tags.
"""

In [234]:
#quick test to see if it's working:
text = "My 6-year-old daughter loves all things creative and artistic. She's always drawing, painting, or making crafts. We're looking for a summer camp around August 1st that would allow her to explore her artistic talents and express her creativity. Could you recommend a camp that would be a good fit for her interests and age?"
camps = "No camps match this request"
result = ask_claude(prompt_template_decomposed.replace("{{INPUT}}",text).replace("{{CAMPS}}",camps),system=prompt_template_decomposed_system)
print (result[1])

<camp>Sorry, we do not have any summer camps that match your request.</camp>
<reason>Based on the list provided, there are no summer camps available that match the requested date and age for your 6-year-old daughter who is interested in creative and artistic activities like drawing, painting, and crafts. The list indicates that no camps match this particular request.</reason>


In [235]:
#create a get results function, but tweaked to accept camps and the input question
#this is very simular to the get_results function used everywhere else in this notebook.
def get_results_decomposed(camps,prompt_template,prompt_template_system, input_output,model,ignore_cache=False):
    '''
    Take a prompt template, and a set of inputs, and generate results for those inputs.
    prompt_template: the template to test, {{INPUT}} where input from the gold standard set should be slotted in.
    prompt_template_system: system prompt for the template to test
    input_output: an array of input/output pairs, human currated "correct" result examples.
    camps: a dict of [test_id] to string of camps that fit the question with that test ID.
    '''
    print ("Generating results to score...")
    prompts = []
    duplicate_check = {}
    for test_id,category,i,o in input_output:
        if i in duplicate_check:
            print ("Error!  The exact same prompt has been added to the gold standard set multiple times!")
            print ("Test ID %s and %s are the same prompt."%(test_id,duplicate_check[i]))
            raise()
        else:
            duplicate_check[i] = test_id
        
        #pull out the date and age information from the camp listing, since we don't need it anymore.
        this_camp = camps[test_id]
        if "<date>" in this_camp:
            this_camp = this_camp[:this_camp.index("<date>")]+this_camp[this_camp.index("</age_group>")+13:]
        prompt = prompt_template.replace("{{INPUT}}",i).replace("{{CAMPS}}",this_camp)
        #print (prompt)
        prompts.append((test_id,prompt))
    
    #send all questions to Claude
    results_to_test = ask_claude_threaded(prompts, system=prompt_template_system,model=model,ignore_cache=ignore_cache)
    return results_to_test

In [236]:
#generate the list of camps that work for each request in the gold standard set.
#this object will be passed to our tweaked generate results function.
camps = {}

for test_id,category,i,o in test_cases_3:
    #get the requested age and date for this camp
    age_date = testID2age_date[test_id]
    this_age = int(age_date.split(',')[0])
    this_date = int(age_date.split(',')[1])
    #find the matching camps
    this_camp = []
    for campID in campID2age_date:
        age_date = campID2age_date[campID]
        age_date = age_date.replace("[",'').replace("]",'').split(',')
        low_age = int(age_date[0])
        high_age = int(age_date[1])
        low_date = int(age_date[2])
        high_date = int(age_date[3])
        
        if this_age>=low_age and this_age<=high_age and this_date>=low_date and this_date<=high_date:
            this_camp.append(campID2camp[campID])
    this_camp = "\n".join(this_camp).strip()
    if len(this_camp)<1:
        this_camp = "No camps match this request."
    camps[test_id] = this_camp

In [287]:
answer_tag = 'camp'
print ("Haiku:")
results_to_test = get_results_decomposed(camps,prompt_template_decomposed,prompt_template_decomposed_system, test_cases, model="haiku")
scores_haiku = evaluate_results(test_cases, results_to_test, test_prompt_template,test_prompt_template_system, "haiku", answer_tag, threshhold = 80, print_out=True)
print ("Sonnet:")
results_to_test = get_results_decomposed(camps,prompt_template_decomposed,prompt_template_decomposed_system, test_cases,model="sonnet", ignore_cache = False)
scores = evaluate_results(test_cases, results_to_test, test_prompt_template,test_prompt_template_system, "sonnet", answer_tag, threshhold = 80, print_out=True)

Haiku:
Generating results to score...
Average response time was 2.0 seconds.
Total cost before scoring: $0.0062
Scoring results...
Total inputs: 25
Total Correct: 21
Accuracy: 84.0 %
Category Scores:
  age: 75.0
  basic: 100.0
  indirect_age: 100.0
  indirect_time: 100.0
  second_pick: 75.0
  time: 50.0
Sonnet:
Generating results to score...
Average response time was 2.47 seconds.
Total cost before scoring: $0.0702
Scoring results...
Total inputs: 25
Total Correct: 25
Accuracy: 100.0 %
Category Scores:
  age: 100.0
  basic: 100.0
  indirect_age: 100.0
  indirect_time: 100.0
  second_pick: 100.0
  time: 100.0


And there we go!  We can get up to 100% accuracy for the entire system by using Sonnet on this step.

In [243]:
#take a peek at the ones Haiku got wrong.
already_done = []
for category, question,correct_answer,prompt_answer,score,reason,passed,prompt_reason in scores_haiku:
    if int(score)>=50:continue
    if category in already_done:continue #look at one example per category
    already_done.append(category)
    print ()
    print ("Category:",category,"Score:",score)
    print ("Question:",question)
    print ("Gold Ans:",correct_answer)
    print ("LLM  Ans:",prompt_answer.strip())
    print ("LLM's reasoning:\n"+prompt_reason)
    print ("_________________")
print ("")


Category: second_pick Score: 0
Question: My 11-year-old daughter is passionate about singing, acting, and dancing. Her dream is to perform in the circus!  She loves clowns so much.  We are looking for a camp during the week of August 11th that would allow her to explore her interests further and potentially participate in a production. Can you recommend a suitable camp for her?
Gold Ans: Musical Theatre Extravaganza
LLM  Ans: Sorry, we do not have any summer camps that match your request.
LLM's reasoning:
The available summer camps do not include any that focus on circus performance or clowning, which are the specific interests mentioned in the request. The closest match is the "Musical Theatre Extravaganza" camp, but it does not cover circus-related activities. Additionally, the requested camp dates of the week of August 11th do not align with the available camps, which run from July 31st to August 11th or August 7th to August 18th.
_________________

Category: time Score: 0
Question

## 2b) Volume based decomposition
Here we'll consider an example use case of a user who would like to undersand how many unique characters are in a novel, and learn a bit about the three most common characters.

Jumping straight to the results, by breaking down this task, it runs around twice as fast, and accuracy improves slightly as well.  (Here accuracy is based on the 12 characters identified in the Frankenstein Cliff notes.)
![results_2](results_example_2.png)

We start by downloading the novel.  For this example we use Frankenstein by Mary Shelley, as it is in public domain.

In [244]:
import requests, re
from bs4 import BeautifulSoup 

In [245]:
#grab the text from the Gutenberg project, a collection of public domain works.
#We use Beautiful Soup to parse the HTML of the webpage.
url = "https://www.gutenberg.org/files/84/84-h/84-h.htm"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
raw_full_text_webpage = soup.text

In [246]:
#Cut the top and bottom of the webpage so that we only have the text of the book.
raw_full_text = raw_full_text_webpage[raw_full_text_webpage.index("Letter 1\n\nTo Mrs. Saville, England."):raw_full_text_webpage.index("*** END OF THE PROJECT GUTENBERG EBOOK FRANKENSTEIN ***")].replace("\r\n"," ").replace("\n", " ")
#encode some misc unicode charaters.
full_text = raw_full_text.encode('raw_unicode_escape').decode()
#show that we found the expected length
words_count = len(full_text.split(" "))
pages_count = int(words_count/500)#quick estimate, real page count is dependant on page and font size.
print ("Approximate word count:",words_count)
print ("Approximate page count:",pages_count)

Approximate word count: 76553
Approximate page count: 153


### Now that we have our novel, let's try to find all the unique characters with a single prompt.

In [279]:
long_prompt_template = """Consider the following novel:
<novel>
{{NOVEL}}
</novel>

How many unique characters are there with at least one spoken line of dialog?  Please also provide a brief description of only the top three most common characters, each in separate paragraphs. 
Only count charaters that have at least one spoken line of dialog.
"""

long_prompt = long_prompt_template.replace("{{NOVEL}}",full_text)

In [280]:
long_response = ask_claude(long_prompt, model="sonnet",ignore_cache=False)
print("Time at Bedrock: ",long_response[3],"sec")
print("Tokens: ",long_response[2]["input_tokens"]+long_response[2]["output_tokens"])
print("Cost: $",round(calculate_cost(long_response[2],"sonnet"),4))
print("Response from model:")
print(long_response[1])

Time at Bedrock:  40.25 sec
Tokens:  98219
Cost: $ 0.2972
Response from model:
Based on the novel, there are 7 unique characters that have at least one spoken line of dialog:

1. Victor Frankenstein: The protagonist and narrator for most of the novel. He is a scientist who creates a hideous but sentient creature, leading to tragic consequences.

2. The Creature/Monster: Frankenstein's creation, who narrates parts of the story. Initially seeking companionship and acceptance, he becomes vengeful and murderous after being rejected by his creator and society.

3. Robert Walton: The explorer who rescues Victor Frankenstein and to whom Frankenstein narrates his story. He has a few spoken lines towards the end of the novel.

4. Elizabeth Lavenza: Victor Frankenstein's adopted sister and fiancée. She has a few spoken lines, including a letter she writes to Victor.

5. Alphonse Frankenstein: Victor Frankenstein's father, who has a few spoken lines.

6. Henry Clerval: Victor Frankenstein's close

### Not bad!  93K tokens processed in about 40 seconds.  Let's see if we can make that faster and cheaper using prompt decomposition.
### We'll divide the novel into thirds, run each third in parallel, then write a fourth prompt to combine the results.

In [253]:
short_prompt_template = """Consider the following portion of a novel:
<novel>
{{NOVEL}}
</novel>

Please provide a list of unique characters, each in a character tag.  Inside the character tag should be a name tag with their name,
a count tag with an exact count of times they appear, and a description tag with a brief description of that character.
Only count charaters that have at least one spoken line of dialog.
"""

#let's cut the novel into thirds.
third = int(len(full_text)/3)
short_prompt_1 = (1,short_prompt_template.replace("{{NOVEL}}",full_text[:third]))
short_prompt_2 = (2,short_prompt_template.replace("{{NOVEL}}",full_text[third:third+third]))
short_prompt_3 = (3,short_prompt_template.replace("{{NOVEL}}",full_text[third+third:]))

### Now let's run these three prompts in parallel

In [283]:
short_responces = ask_claude_threaded([short_prompt_1,short_prompt_2,short_prompt_3],model='sonnet',ignore_cache=False)
time_1 = short_responces[0][1][3]
time_2 = short_responces[1][1][3]
time_3 = short_responces[2][1][3]
average_time = round((time_1+time_2+time_3)/3,2)
print("Average time at Bedrock: ",average_time,"sec")
total_cost_shorts = round(calculate_cost(short_responces[0][1][2],"sonnet")+calculate_cost(short_responces[1][1][2],"sonnet")+calculate_cost(short_responces[2][1][2],"sonnet"),4)
print("Total Cost: $",total_cost_shorts)

#show the reply from one of the three prompts
print("Example Output:")
print(short_responces[0][1][1][:700]+" ...")

Average time at Bedrock:  15.95 sec
Total Cost: $ 0.3136
Example Output:
Here is a list of unique characters with their names, counts, and descriptions, for characters that have at least one spoken line of dialog:

<character>
  <name>Victor Frankenstein</name>
  <count>1</count>
  <description>The narrator and protagonist, a scientist who creates a hideous monster.</description>
</character>

<character>
  <name>Elizabeth Lavenza</name>
  <count>2</count>
  <description>Victor's cousin and adopted sister, who he loves.</description>
</character>

<character>
  <name>Alphonse Frankenstein</name>
  <count>1</count>
  <description>Victor's father.</description>
</character>

<character>
  <name>Justine Moritz</name>
  <count>2</count>
  <description>A kind servant  ...


### So far it's looking good!  We've processed the whole novel in around 17 seconds, down from 42.  Let's make a final call to get a final result that matches our original long prompt.

In [261]:
final_prompt_template = """Consider the following list of charaters from a novel.  Each entry contains the character's name,
a count of the number of times they appeared, and a brief description of that charater:
<characters>
{{CHARACTERS}}
</characters>
Some charaters may be listed more than once.  Use the name and description to determine that two entries are the same, 
and if they are, sum their count to support your responce.

How many unique characters are there?  Please also provide a brief description of the top three most common characters in separate paragraphs. 
"""

characters = short_responces[0][1][1]+short_responces[1][1][1]+short_responces[2][1][1]

final_prompt = final_prompt_template.replace("{{CHARACTERS}}",characters)

In [284]:
final_response = ask_claude(final_prompt, model="sonnet")
time_final_step = final_response[3]
print("Final step time was %s seconds."%final_response[3])
print("Total time for decomposed task was %s seconds."%(round(final_response[3]+average_time,2)))
print("Total cost for decomposed task was $%s."%(total_cost_shorts+round(calculate_cost(final_response[2],"sonnet"),4)))
print("Final response from decomposed task:")
print(final_response[1])

Final step time was 4.06 seconds.
Total time for decomposed task was 20.01 seconds.
Total cost for decomposed task was $0.3215.
Final response from decomposed task:
Based on the provided list of characters, there are 9 unique characters:

1. Victor Frankenstein
2. The creature/monster/daemon/fiend
3. Elizabeth Lavenza
4. Henry Clerval
5. Alphonse Frankenstein
6. Justine Moritz
7. M. Waldman
8. M. Krempe
9. William Frankenstein

The top three most common characters are:

Victor Frankenstein: The protagonist and narrator, a young scientist who creates a hideous sapient creature in an unorthodox scientific experiment. He appears many times throughout the novel as the central character.

The creature/monster/daemon/fiend: The hideous but intelligent being created by Victor Frankenstein, who demands that Victor create a female companion for him. He appears many times as the antagonist, seeking revenge against his creator for abandoning him.

Robert Walton: The explorer who rescues Frankenst

### Results: Decomposition is twice as fast!
This final prompt took about 5 seconds to run.  The original long prompt took 42 seconds to run, and our decomposed version took 17s + 5, or 22 seconds total.  Almost twice as fast to do the same amount work!
Note that the decomposed version actually found more characters.  It is somewhat common that the quality will improve with smaller, more focused prompts, because the LLM can focus more when the prompt is smaller.  Now go forth and decompose!

![Decompose](decompose.png)